In [1]:
from itertools import combinations
from tqdm.auto import tqdm
import numpy as np
import sample as sampler
import utils
import math
import attacker

Implement the attack by simulating data.
1. generate proper data with given error rate as a puzzle

## Generate puzzle with given error_rate

generate n isometrices(sketches) with noise. Assume the original template is w, then the output for each sketching is Matrix = SS(w + noise) and the correponding codeword as b satisfying Matrix * (w + noise) = b.

In [ ]:
# codeword parameters
dimension = 512
alpha = 8   # if we want the whole attack's time is acceptable, you need to set alpha <= 8 and the noise small
            # else if we assume the sampled matrix by attack is always "correct", it does not matter
# number of secure sketches from the same biometric source
n = 511
error_rate = 0.0    # angle(w + noise_1, w + noise_2) 
                    # approximate 2*arcsin(error_rate * \sqrt(2) / \sqrt(1 + error_rate^2) / 2)
print("Estimated angle between two noisy templates: {}".format(2 * np.arcsin(error_rate * np.sqrt(2) / np.sqrt(1 + error_rate**2) / 2) * 180 / np.pi))

In [ ]:
w, cs, isometric_matrixes, _ = sampler.generate_puzzle_n(dimension, alpha, error_rate=error_rate, n = n)

In [ ]:
# get the mean angle between each pair of templates, i.e. angle(w + noise_1, w + noise_2)

angles = []
ws = [M.T @ b for M, b in zip(isometric_matrixes, cs)]
for i, j in tqdm(combinations(range(len(ws)), 2)):
    angles.append(utils.get_angle_of_two_vectors(ws[i], ws[j]))

print("Test Mean Degree: {:.2f}{}".format(np.mean(angles), u'\u00b0'))

In [ ]:
# p_array = [0 for _ in range(n+1)]
# p_array[0] = 1


# def tmp_subset_n_k(n, k):
#     result = 1
#     for i in range(k):
#         result = result * (n - i)
#     for i in range(k):
#         result = result // (i + 1)
#     return result

# def tmp_calculate(n, alpha, x, m):
#     return tmp_subset_n_k(n-(x-m), m) * tmp_subset_n_k(x-m, alpha-m)
    

# for _ in (pbar:=tqdm(range(n))):
#     tmp_p_array = [0 for _ in range(n+1)]
#     for x in range(alpha, n+1):
#         result = 0
#         for m in range(0, alpha+1):
#             result += tmp_calculate(n, alpha, x, m) * p_array[x-m]
#         tmp_p_array[x] = result
#     # pbar.set_postfix({"n th element": p_array[n-1]})
#     p_array = tmp_p_array

# result = p_array[n]
# result = result / (tmp_subset_n_k(n, alpha) ** (n))

In [ ]:
# print(1-result)
# print(-np.log2(1-result) + np.log2(n))
# print(1 / (1 - result))

## solving original template if sampled matrix is correct

ensure the matrix generated by linear equation sampler is "correct" as in Definition 4.1 in paper.

svd solver

In [ ]:
# k_each_matrix should be proper setting with constraint k_each_matrix * len(isometric_matrixes) >= dimension - 1
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, cs, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="SVD", disable_tqdm=False, k_each_matrix=1)

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

lsa solver

In [ ]:
# k_each_matrix should be proper setting with constraint k_each_matrix * (len(isometric_matrixes) -1) ~ dimension, see proper settings in paper's experiments
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, cs, dimension, alpha, threshold=49, max_rtimes=10000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, error_rate = error_rate * 3.0)

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

## solving original template without preposition that sampled matrix is "correct"

In [ ]:
tmpr = (2 ** (utils.subset_n_a_alpha(dimension, alpha, 1) * 1) ) ** len(isometric_matrixes)
print("esimate running times: ", tmpr)
print("log2 esimate running times: ", np.log2(tmpr))

In [ ]:
# toy example for small k = 2, larger k needs exponential time
# k_each_matrix should be proper setting with constraint k_each_matrix * len(isometric_matrixes) >= dimension - 1
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000,
                                                        algorithm="SVD", disable_tqdm=False, k_each_matrix=1, scale=100.0, partition=False)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

In [ ]:
# toy example for small k = 2
# k_each_matrix should be proper setting with constraint k_each_matrix * (len(isometric_matrixes) -1) ~ dimension, see proper settings in paper's experiments
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, scale=1000.0, error_rate=error_rate * 3.0, partition=True)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))